In [1]:
import cv2
import numpy as np
import scipy
import scipy.ndimage

# Load Video
cap = cv2.VideoCapture('vid.mp4')


fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi',fourcc, 20.0, (160,120))

# Resize frame ke 160x120 px
while True:
    ret, frame = cap.read()
    if ret == True:
        b = cv2.resize(frame,(160,120),fx=0,fy=0, interpolation = cv2.INTER_CUBIC)
        out.write(b)
    else:
        break
    
cap.release()
out.release()
cv2.destroyAllWindows()

cap = cv2.VideoCapture('output.avi')

countFrame = 0
detectFrame = 0

# intensities to be considered 'skin'
lower = np.array([0, 48, 80], dtype = "uint8")
upper = np.array([20, 255, 255], dtype = "uint8")

                  
while(True):
    ret, img = cap.read()        
    countFrame += 1
    if ret:
        # INFORMASI FITUR MATA
        # rgb to gray
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#         cv2.imshow('Grayscale', gray)
        
        # minimum filter
        mf = scipy.ndimage.filters.minimum_filter(gray,3)
#         cv2.imshow('Minimum Filter', mf)
        
        # threshold binary
        _, th = cv2.threshold(mf,127,255,cv2.THRESH_BINARY)
#         cv2.imshow('Iteratif Threshold', th)
        
        # TEMPLATE WAJAH
        # Skin Selection
        converted = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        skinMask = cv2.inRange(converted, lower, upper)
#         cv2.imshow("Skin Selection", skinMask)
        
        # Gaussian Filter
        gb = cv2.GaussianBlur(skinMask, (3, 3), 0)
#         gb = scipy.ndimage.gaussian_filter(skinMask, sigma=3)
#         cv2.imshow("Gaussian Filter", gb)

        # Adaptive Threshold
#         gb = scipy.ndimage.gaussian_filter(skinMask, sigma=3)
        at = cv2.adaptiveThreshold(gb, 255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 3, 2)
#         cv2.imshow("Adaptive Filter", at)
        
        
        # Bitwise xor
        xor = cv2.bitwise_xor(th, at)
#         cv2.imshow("Bitwise XOR", xor)
        
        # Gaussian Filter
        gf = cv2.GaussianBlur(xor, (3, 3), 0)
        _, lth = cv2.threshold(gf,127,255,cv2.THRESH_BINARY)
#         cv2.imshow("Hasil", lth)
        
        # Iteratif Threshold
        im2, ctrs, hier = cv2.findContours(lth.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 
        
        # Pemetaan Array deteksi mata
        sorted_ctrs = sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])
        for i, ctr in enumerate(sorted_ctrs): 
            # Get bounding box 
            x, y, w, h = cv2.boundingRect(ctr) 

            # Getting ROI 
            roi = img[y:y+h, x:x+w] 
            # show ROI 
            #cv2.imshow('segment no:'+str(i),roi) 
            cv2.rectangle(img,(x,y),( x + w, y + h ),(0,255,0),2) 
   
        cv2.imshow('frame',img)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    else:
        break
        
cap.release()
cv2.destroyAllWindows()

print(countFrame)
print(detectFrame)

457
0
